In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import random
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
import tensorflow as tf
import pandas as pd
from src import utils
from src.features.load_dataset import import_stamp_yoochoose
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import os
import shutil
import datetime

from tensorflow import keras
from tqdm.notebook import tqdm
from collections import defaultdict
import functools
import copy
from itertools import chain
from src.models import TestModel
from src.models import stmp
from src.metrics import MrrMetric

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(f"tensorflow version: {tf.__version__}")

tensorflow version: 2.2.0


In [2]:
train_data, val_data, test_data, idx2item, embedding_sizes = import_stamp_yoochoose(64)

X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data

num_x_train = len(X_train)
n_items, max_session_len = embedding_sizes
max_embedding_key = n_items

print(f"n_items: {n_items}")
print(f"max_session_len: {max_session_len}")

using last item in session as class
using last item in session as class
train_val_test_split @ 0.8 train, 0.2 val, 0 test
n_items: 17695
max_session_len: 146


In [11]:
# config = TestModel.default_config(n_items, max_session_len)
# model = TestModel.model(config=config)

config = stmp.default_config(n_items)
model = stmp.model(config=config)

train_metrics = {
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
}
val_metrics = {
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
}
test_metrics = {
    "acc": tf.keras.metrics.SparseCategoricalAccuracy(),
    "top5 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5),
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
    "top20 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20),
    "mrr5": MrrMetric(k=5),
    "mrr10": MrrMetric(k=10),
    "mrr20": MrrMetric(k=20)
}

train_metrics_rec = defaultdict(lambda: [])
val_metrics_rec = defaultdict(lambda: [])
test_metrics_rec = defaultdict(lambda: [])

current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
print(f"starting experiment '{current_time}'")
expr_dir = Path("logs")/current_time
train_log_dir = expr_dir/"train"
val_log_dir = expr_dir/"val"
test_log_dir = expr_dir/"test"

os.makedirs(expr_dir)
for tensorboard_dir in [train_log_dir, val_log_dir, test_log_dir]:
    os.makedirs(tensorboard_dir, exist_ok=True)

train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
val_summary_writer = tf.summary.create_file_writer(str(val_log_dir))
model_saver = utils.TopModelSaver("models/last_run_weights.h5py")

print(f"batch_size: {config['batch_size']}")
for epoch in range(config["epochs"]):
    print(f"epoch {epoch}")
    # Reset the metrics at the start of the next epoch
    for metric in utils.flatten([train_metrics.values(), val_metrics.values(), test_metrics.values()]):
        metric.reset_states()

    for i, (X, y) in tqdm(list(enumerate(utils.batchify(X_train, y_train, shuffle=True, batch_size=config["batch_size"])))):
        X = [sess[::-1] for sess in X] # reversing sessions so most recent item is last
        X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
        preds, loss = model.train_step(tf.constant(X), tf.constant(mask), tf.constant(y), train_metrics.values()) 
        if np.isnan(loss):
            print("LOSS IS NAN")
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', loss.numpy(), step=epoch*(int(num_x_train/config["batch_size"])+1)+i)
            
    for X, y in tqdm(list(utils.batchify(X_val, y_val, shuffle=True, batch_size=config["batch_size"]))):
        X = [sess[::-1] for sess in X]
        X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
        preds = model.test_step(tf.constant(X), tf.constant(mask), tf.constant(y), val_metrics.values())
    with val_summary_writer.as_default():
        tf.summary.histogram("preds", preds, step=epoch) # intentionally only getting the last preds cause it takes a while

    with train_summary_writer.as_default():
        for label, metric in train_metrics.items():
            train_metrics_rec[label].append(metric.result())
            print(f"{label} train: {metric.result()}")
            tf.summary.scalar(label, metric.result(), step=epoch)
    with val_summary_writer.as_default():
        for label, metric in val_metrics.items():
            val_metrics_rec[label].append(metric.result())
            print(f"{label} val: {metric.result()}")
            tf.summary.scalar(label, metric.result(), step=epoch)

    model_saver.save_best(model, val_metrics["top10 acc"].result())

starting experiment '2020-07-07_16:44:21'
batch_size: 1024
epoch 0



top10 acc train: 0.10173007845878601
top10 acc val: 0.11157683283090591
new best score: 0.11157683283090591; saving weights @ models/last_run_weights.h5py
epoch 1



top10 acc train: 0.10196062177419662
top10 acc val: 0.08089529722929001
best score remains 0.11157683283090591; not saving weights
epoch 2



top10 acc train: 0.08252208679914474
top10 acc val: 0.08751781284809113
best score remains 0.11157683283090591; not saving weights
epoch 3



top10 acc train: 0.08731098473072052
top10 acc val: 0.08437421172857285
best score remains 0.11157683283090591; not saving weights
epoch 4


...


top10 acc train: 0.7206509709358215
top10 acc val: 0.400285005569458
best score remains 0.40372201800346375; not saving weights
epoch 496



top10 acc train: 0.7223066091537476
top10 acc val: 0.4002012014389038
best score remains 0.40372201800346375; not saving weights
epoch 497



top10 acc train: 0.7230611443519592
top10 acc val: 0.4002012014389038
best score remains 0.40372201800346375; not saving weights
epoch 498



top10 acc train: 0.7237422466278076
top10 acc val: 0.4010394811630249
best score remains 0.40372201800346375; not saving weights
epoch 499



top10 acc train: 0.7242557406425476
top10 acc val: 0.4009556472301483
best score remains 0.40372201800346375; not saving weights


In [12]:
# config = stmp.default_config(n_items)
# model = stmp.model(config=config)
model.load_weights(model_saver.location)

for metric in test_metrics.values():
    metric.reset_states()

preds, ys = [], []
for X, y in tqdm(list(utils.batchify(X_test, y_test, shuffle=True, batch_size=config["batch_size"]))):
    X = [sess[::-1] for sess in X]
    X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
    ys.extend(y)
    preds.extend(model.test_step(tf.constant(X), tf.constant(mask), tf.constant(y), test_metrics.values()).numpy())

# brute force verify acc
acc, acc5, acc10, acc20 = 0, 0, 0, 0
for y, pred in tqdm(list(zip(ys, preds))):
    argsorted_preds = (-pred).argsort()
    if y in argsorted_preds[:20]:
        acc20 += 1

        if y in argsorted_preds[:10]:
            acc10 += 1

            if y in argsorted_preds[:5]:
                acc5 += 1

                if y == argsorted_preds[0]:
                    acc += 1

print("brute force sanity check acc", acc/len(ys))
print("brute force sanity check top5 acc", acc5/len(ys))
print("brute force sanity check top10 acc", acc10/len(ys))
print("brute force sanity check top20 acc", acc20/len(ys))

for label, metric in test_metrics.items():
    print(label, metric.result())


brute force sanity check acc 0.0868572174367006
brute force sanity check top5 acc 0.2728399895588619
brute force sanity check top10 acc 0.37770817019055075
brute force sanity check top20 acc 0.48146697990080917
acc tf.Tensor(0.08731402, shape=(), dtype=float32)
top5 acc tf.Tensor(0.2950274, shape=(), dtype=float32)
top10 acc tf.Tensor(0.38540852, shape=(), dtype=float32)
top20 acc tf.Tensor(0.48172802, shape=(), dtype=float32)
mrr5 tf.Tensor(0.14712977, shape=(), dtype=float32)
mrr10 tf.Tensor(0.16132635, shape=(), dtype=float32)
mrr20 tf.Tensor(0.16877256, shape=(), dtype=float32)


In [201]:
a = np.arange(6).reshape(2, 3)
b = np.arange(6).reshape(2, 3)
c = np.arange(12).reshape(4, 3)

print(a)
print(b)
print(c)
print(utils.trilinear_product(a, b, c))

[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
tf.Tensor(
[[  9  24  39  54]
 [ 66 216 366 516]], shape=(2, 4), dtype=int64)


In [27]:
inputs = [[3, 1, 2, 3], [1, 2, 3]]
inputs = [np.array(sess) for sess in inputs]
padded_inputs, mask = utils.mask_length(inputs, justify="right")

X_embeddings = tf.nn.embedding_lookup(tf.constant([[0, 0], [1, 1], [2, 2], [3, 3]], dtype=tf.float32), padded_inputs)
X_s = X_embeddings[:, :-1]
ms = utils.masked_mean(X_s, mask[:, :-1, None])
mt = X_embeddings[:, -1]

print(ms)

tf.Tensor(
[[2 2]
 [1.5 1.5]], shape=(2, 2), dtype=float32)


In [26]:
sessions = [
    [[0], [1]],
    [[1], [2], [3]]
]
sessions = [np.array(sess) for sess in sessions]

utils.mask_length(sessions)

(array([[[0],
         [0],
         [0],
         [1]],
 
        [[0],
         [1],
         [2],
         [3]]]),
 array([[0, 0, 1, 1],
        [0, 1, 1, 1]], dtype=float32))

In [60]:
# config = TestModel.default_config(n_items=4, embedding_size=3, max_session_len=3)
# model = TestModel.model(config=config)

config = stmp.default_config(n_items=5, embedding_size=3, learning_rate=.00000001)
model = stmp.model(config=config)

sessions = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]], dtype=np.int32)
labels = np.array([1, 2, 3], dtype=np.int32)
sessions, mask = utils.mask_length(sessions, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
print("sessions", sessions)
print("mask", mask)

for i in range(3):
    print("epoch", i)
    print("pred", model.train_step(tf.constant(sessions), tf.constant(mask), tf.constant(labels), [])[1])
    print("embs", model.item_embs)

sessions [[0 1 1 1]
 [0 2 2 2]
 [0 3 3 3]]
mask [[0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]]
epoch 0
hs_val tf.Tensor(
[[-5.15e-06 -8.15e-05 -2.67e-05]
 [-7.32e-05 -2.18e-05 -4.38e-05]
 [6.04e-05 0.000156 8.73e-05]], shape=(3, 3), dtype=float32)
ht_val tf.Tensor(
[[-0.000153 -7.05e-05 1.82e-05]
 [4.13e-05 -4.16e-06 1.97e-06]
 [-0.000113 4.56e-05 -0.000115]], shape=(3, 3), dtype=float32)
trilinear_product tf.Tensor(
[[0 -6.63e-12 -6.34e-12 1.21e-12 -6.94e-12]
 [0 5.07e-12 -4.98e-13 -2.3e-12 -1.3e-12]
 [0 1.14e-11 -1.23e-11 1.56e-11 1.9e-11]], shape=(3, 5), dtype=float32)
sigmoid_fc [<tf.Variable 'dense_128/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[0.644, 0.262, 0.288, 0.764, -0.392],
       [0.55, -0.509, 0.343, 0.74, -0.364],
       [-0.434, 0.0031, 0.53, -0.0639, -0.335],
       [-0.761, -0.632, 0.355, -0.165, -0.139],
       [0.298, -0.386, 0.0566, 0.221, 0.532]], dtype=float32)>, <tf.Variable 'dense_128/bias:0' shape=(5,) dtype=float32, numpy=array([0, 0, 0, 0, 0], dtype=float32)>]


In [52]:
preds = [[1, 2, 3, 4], [1, 2, 3], []]
preds = [np.array(sess, dtype=np.float32) for sess in preds]

padded_preds, mask = utils.mask_length(preds)
print(utils.masked_mean(padded_preds, mask, axis=1))

ValueError: could not broadcast input array from shape (0) into shape (4)

In [40]:
@tf.function()
def graph(a): # in tensorflow graph mode
    print(f"in eager mode: {tf.executing_eagerly()}")
    tf.print(a * 0)

def eager(a): # in tensorflow eager mode
    print(f"in eager mode: {tf.executing_eagerly()}")
    tf.print(a * 0)

graph(tf.constant(np.nan))
eager(tf.constant(np.nan))

# wow optimizations

in eager mode: False
0
in eager mode: True
nan


In [112]:
import os
os.mkdir("58_acc_76_top5_85_top50.tf_save")
model.save_weights("58_acc_76_top5_85_top50.tf_save/model")

AttributeError: 'NoneType' object has no attribute 'replace'

In [6]:
# record experiment scores
import datetime

with open("last_expr_name.txt", "r+") as last_expr_name:
    try:
        last_expr_name = int(last_expr_name.read())+1
    except:
        last_expr_name = 0
with open("last_expr_name.txt", "w") as expr_names:
    expr_names.write(str(last_expr_name))

experiment_name = f"expr{last_expr_name}_{datetime.date.today()}"

import os
expr_dir = Path("records")/experiment_name
(expr_dir/"model").mkdir(parents=True, exist_ok=True)

print("saving at", str(expr_dir/"model/model"))
# model.save_weights(str(expr_dir/"model/model"))

topn_labels = ["acc", "top5 acc", "top10 acc", "top20 acc", "top50 acc"]
train_accs = [train_accs_rec[label] for label in topn_labels]
test_accs = [test_accs_rec[label] for label in topn_labels]
for label, train_acc, test_acc in zip(topn_labels, train_accs, test_accs):
    kwargs = {
        "title": f"test {label}",
        "xlabel": "epoch",
        "ylabel": label
    }
    save_accs(expr_dir/"scores", [test_acc], kwargs)
    
    kwargs = {
        "title": f"train {label}",
        "xlabel": "epoch",
        "ylabel": label
    }
    save_accs(expr_dir/"scores", [train_acc], kwargs)
    
kwargs = {
    "title": f"test accs",
    "xlabel": "epoch",
    "ylabel": "acc",
    "legend": topn_labels
}
save_accs(expr_dir/"scores", test_accs, kwargs)


with open(expr_dir/"scores"/"raw_scores.pickle", "wb+") as raw_scores_handle:
    scores = {}
    for label in topn_labels:
        scores[f"train {label}"] = train_accs_rec[label]
        scores[f"test {label}"] = test_accs_rec[label]
    pickle.dump(scores, raw_scores_handle)
    


SyntaxError: invalid syntax (<ipython-input-6-08255273a46b>, line 55)